In [1]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
import numpy as np
import pandas_profiling

# library untuk eksplorasi data
import matplotlib.pyplot as plt
import seaborn as sns

# library untuk membagi data
from sklearn.model_selection import train_test_split

# library untuk evaluasi hasil prediksi
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import average_precision_score

# library untuk bebrapa model
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost.sklearn import XGBClassifier

from sklearn.feature_selection import SelectFromModel

# library untuk tunning parameter
from sklearn.model_selection import GridSearchCV, StratifiedKFold

# library untuk menggabungkan beberapa algoritme
from imblearn.pipeline import Pipeline
# library untuk menyeimbangkan data
from imblearn.over_sampling import SMOTE

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder, StandardScaler, KBinsDiscretizer
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.compose import ColumnTransformer
# import category_encoders as ce

from sklearn.calibration import CalibratedClassifierCV

C:\Users\muhyi\AppData\Local\Continuum\anaconda3\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
C:\Users\muhyi\AppData\Local\Continuum\anaconda3\lib\site-packages\dask\dataframe\utils.py:13: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
C:\Users\muhyi\AppData\Local\Continuum\anaconda3\lib\site-packages\distributed\config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [3]:
pd.set_option('display.max_columns', None)

In [4]:
df_train = pd.read_csv('train_prep.csv')
df_test = pd.read_csv('test_prep.csv')

# Data Prepaparation

In [5]:
df_train_prep1 = df_train.copy()
df_test_prep1 = df_test.copy()

In [6]:
new_var_one_hot = ['age2','age3','year_graduated2','in study','person level up','sick_vs_leaves']

one_hot_var = [
    'job_level_oh', 
    'person_level_oh', 
    'Employee_type', 
    'Education_level',
#     'Employee_status',   
    'gender', 
    'marital_status_maried(Y/N)'] + new_var_one_hot

ordinal_val = [
    'job_level',
    'person_level',
    'Education_level'
#     'achievement_target_1', 
#     'achievement_target_2'
]

new_var_num = ['year_graduated','total leave','total_rotation',
               'age when graduated','age when graduated2','papa muda',
               'papa muda per tanggungan','age per dependencies']

numeric_var = [
    'job_duration_in_current_job_level',   
    'job_duration_in_current_person_level',
    'job_duration_in_current_branch', 
    'age', 
    'number_of_dependences',
    'GPA',   
    'branch_rotation', 
    'job_rotation', 
    'assign_of_otherposition',   
    'annual leave', 
    'sick_leaves',
    'Last_achievement_%', 
    'Achievement_above_100%_during3quartal'] + new_var_num

In [7]:
# ordinal_mapping = [
#     {'col':'job_level',
#     'mapping':{None:0,'JG04':1,'JG05':2,'JG03':3,'JG06':4}},
#     {'col':'person_level',
#     'mapping':{None:0,'PG01':1,'PG02':2,'PG03':3,'PG04':4, 'PG05':5, 'PG06':6, 'PG07':7, 'PG08':8}},
#     {'col':'Education_level',
#     'mapping':{None:0,'level_0':1,'level_1':2,'level_2':3, 'level_3':4, 'level_4':5, 'level_5':6}}    
#     ]

# ordinal_encoder = ce.OrdinalEncoder(cols = ['job_level','person_level','Education_level'],mapping = ordinal_mapping)

one_hot_encoder_pipeline = Pipeline([
                                    ('one hot encoder',OneHotEncoder(handle_unknown = 'ignore'))
])

# numerical_pipeline = SimpleImputer(strategy = 'constant',fill_value = 0)

numerical_pipeline = Pipeline([
                                ('imputer',SimpleImputer(strategy = 'constant',fill_value = 0)),
                                ('bin',KBinsDiscretizer(encode = 'onehot',strategy = 'kmeans',n_bins=5))
])

transformer = ColumnTransformer([
    ('one hot encoder',one_hot_encoder_pipeline,one_hot_var),
#     ('ordinal encoder',ordinal_encoder,ordinal_val),
    ('numerical_pipeline',numerical_pipeline,numeric_var)
])

In [8]:
# ?KBinsDiscretizer

# Data Splitting

In [9]:
var_x = numeric_var + one_hot_var #+ ordinal_val

X = df_train_prep1[var_x]
X_sub = df_test_prep1[var_x]
y = df_train_prep1['Best Performance']

In [10]:
# seed = 1995
seed = 70621

X_trainval, X_test, y_trainval, y_test = train_test_split(X,y, stratify = y, test_size = 1500, random_state = seed)
X_train, X_val, y_train, y_val = train_test_split(X_trainval,y_trainval, stratify = y_trainval, test_size = 1500, random_state = seed)

> ## Model (Benchmark)

In [11]:
model = LogisticRegression(solver = 'newton-cg')

estimator = Pipeline([
                      ('preprocess',transformer),
                      ('clf',model)
])

In [12]:
estimator.fit(X_train, y_train)

C:\Users\muhyi\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\_discretization.py:189: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  centers = km.fit(column[:, None]).cluster_centers_[:, 0]
C:\Users\muhyi\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\_discretization.py:202: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 12 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.' % jj)


Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('one hot encoder',
                                                  Pipeline(steps=[('one hot '
                                                                   'encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['job_level_oh',
                                                   'person_level_oh',
                                                   'Employee_type',
                                                   'Education_level', 'gender',
                                                   'marital_status_maried(Y/N)',
                                                   'age2', 'age3',
                                                   'year_graduated2',
                                                   'in study',
                                                   'person level up'

In [13]:
y_prob_val = estimator.predict_proba(X_val)[:,1]
y_pred_val = np.where(y_prob_val > 0.13, 1, 0)

print('auc',roc_auc_score(y_val,y_prob_val),'\n')
print('classification report \n', classification_report(y_val,y_pred_val))

auc 0.5545703125 

classification report 
               precision    recall  f1-score   support

           0       0.88      0.45      0.59      1280
           1       0.17      0.64      0.26       220

    accuracy                           0.47      1500
   macro avg       0.52      0.54      0.43      1500
weighted avg       0.77      0.47      0.54      1500



In [14]:
y_prob_train = estimator.predict_proba(X_train)[:,1]
y_pred_train = np.where(y_prob_train > 0.13, 1, 0)

print('auc',roc_auc_score(y_train,y_prob_train),'\n')
print('classification report \n', classification_report(y_train,y_pred_train))

auc 0.6209941544488017 

classification report 
               precision    recall  f1-score   support

           0       0.90      0.47      0.62      6956
           1       0.18      0.70      0.29      1197

    accuracy                           0.50      8153
   macro avg       0.54      0.58      0.45      8153
weighted avg       0.80      0.50      0.57      8153



In [15]:
y_prob_test = estimator.predict_proba(X_test)[:,1]
y_pred_test = np.where(y_prob_test > 0.13, 1, 0)

print('auc',roc_auc_score(y_test,y_prob_test),'\n')
print('classification report \n', classification_report(y_test,y_pred_test))

auc 0.5917933238636364 

classification report 
               precision    recall  f1-score   support

           0       0.90      0.46      0.61      1280
           1       0.18      0.70      0.29       220

    accuracy                           0.49      1500
   macro avg       0.54      0.58      0.45      1500
weighted avg       0.79      0.49      0.56      1500



> ## Model Selection

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.svm import SVC

In [17]:
model_xgb = XGBClassifier(
    learning_rate = 0.05,
    n_estimators = 1000,
    max_depth = 5,
    min_child_weight = 4,
    gamma= 30,
    subsample = 0.7,
    colsample_bynode = 0.4,
    reg_alpha= 0.005,
    objective= 'binary:logistic',
    n_jobs = -1,
    seed=27)

# model_xgb = CalibratedClassifierCV(model)

model_logreg = LogisticRegression(solver = 'newton-cg')
model_knn = KNeighborsClassifier()
model_dtc = DecisionTreeClassifier()
model_rf = RandomForestClassifier()
# model_svc = SVC(kernel = 'rbf')

estimator = Pipeline([
                      ('preprocess',transformer),
                      ('clf',model)
])

skfold = StratifiedKFold(n_splits = 5)

hyperparam_space = {
    'clf':[model_xgb, model_logreg, model_knn, model_dtc, model_rf]
}

grid_search_model = GridSearchCV(
    estimator, # model to tune
    param_grid = hyperparam_space, # hyperparameter space
    cv = skfold, # evaluation method
    scoring = 'roc_auc', # metrics
    n_jobs = -1 # use all cores
)

In [18]:
grid_search_model.fit(X_trainval, y_trainval)

C:\Users\muhyi\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\_discretization.py:189: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  centers = km.fit(column[:, None]).cluster_centers_[:, 0]
C:\Users\muhyi\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\_discretization.py:202: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 12 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.' % jj)


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             estimator=Pipeline(steps=[('preprocess',
                                        ColumnTransformer(transformers=[('one '
                                                                         'hot '
                                                                         'encoder',
                                                                         Pipeline(steps=[('one '
                                                                                          'hot '
                                                                                          'encoder',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         ['job_level_oh',
                                                                          'person_level_oh',
            

In [19]:
print(grid_search_model.best_params_)
print(grid_search_model.best_score_)

{'clf': LogisticRegression(solver='newton-cg')}
0.5627476379565657


In [20]:
pd.DataFrame(grid_search_model.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,12.782209,0.071982,0.040093,0.003646,"XGBClassifier(base_score=None, booster=None, c...","{'clf': XGBClassifier(base_score=None, booster...",0.539521,0.556744,0.566059,0.553818,0.545408,0.552310,0.009189,2
1,3.069987,0.117769,0.032114,0.002554,LogisticRegression(solver='newton-cg'),{'clf': LogisticRegression(solver='newton-cg')},0.551582,0.548659,0.575032,0.571703,0.566762,0.562748,0.010681,1
2,0.819408,0.185713,2.683422,0.176673,KNeighborsClassifier(),{'clf': KNeighborsClassifier()},0.499464,0.490876,0.527054,0.524145,0.512513,0.510811,0.013933,5
3,1.799987,0.150465,0.046675,0.010543,DecisionTreeClassifier(),{'clf': DecisionTreeClassifier()},0.511114,0.517631,0.529170,0.495092,0.508050,0.512211,0.011212,4
4,7.827462,0.325266,0.065226,0.010337,RandomForestClassifier(),{'clf': RandomForestClassifier()},0.533023,0.519622,0.565898,0.532743,0.539550,0.538167,0.015302,3


> ## Hyperparameter Tuning Logistic

In [21]:
one_hot_encoder_pipeline = Pipeline([
                                    ('one hot encoder',OneHotEncoder(handle_unknown = 'ignore'))
])

# numerical_pipeline = SimpleImputer(strategy = 'constant',fill_value = 0)

numerical_pipeline = Pipeline([
                                ('imputer',SimpleImputer(strategy = 'constant',fill_value = 0)),
                                ('bin',KBinsDiscretizer(encode = 'onehot',strategy = 'uniform'))
])

transformer = ColumnTransformer([
    ('one hot encoder',one_hot_encoder_pipeline,one_hot_var),
#     ('ordinal encoder',ordinal_encoder,ordinal_val),
    ('numerical_pipeline',numerical_pipeline,numeric_var)
])

In [22]:
# ?SelectFromModel

In [23]:
model = LogisticRegression(solver = 'newton-cg')

select = SelectFromModel(model, threshold="median")

estimator = Pipeline([
                      ('preprocess',transformer),
                      ('select',select),
                      ('clf',model)
])

skfold = StratifiedKFold(n_splits = 4)

hyperparam_space = [{
    'preprocess__numerical_pipeline__bin__n_bins':[3,4,5,6],
    'preprocess__numerical_pipeline__bin__encode':['ordinal','onehot'],
    'preprocess__numerical_pipeline__bin__strategy':['uniform','quantile','kmeans'],
    'select__threshold':['0.5*median','0.75*median','median','1.25*median','1.5*median'],
    'clf__C':[0.001,0.01,0.1,1,10,100,1000]},
    {'preprocess__numerical_pipeline__bin__n_bins':[3,4,5,6],
    'preprocess__numerical_pipeline__bin__encode':['ordinal','onehot'],
    'preprocess__numerical_pipeline__bin__strategy':['uniform','quantile','kmeans'],
    'select':[None],
    'clf__C':[0.001,0.01,0.1,1,10,100,1000]}
]

grid_search_simple = GridSearchCV(
    estimator, # model to tune
    param_grid = hyperparam_space, # hyperparameter space
    cv = skfold, # evaluation method
    scoring = 'roc_auc', # metrics
    n_jobs = -1 # use all cores
)

In [ ]:
grid_search_simple.fit(X_trainval, y_trainval)

In [ ]:
grid_search_simple.best_params_

In [ ]:
grid_search_simple.best_score_

In [ ]:
estimator = grid_search_simple.best_estimator_
estimator.fit(X_trainval,y_trainval)

y_prob_test = estimator.predict_proba(X_test)[:,1]
y_pred_test = np.where(y_prob_test > 0.13, 1, 0)

print('auc',roc_auc_score(y_test,y_prob_test),'\n')
print('classification report \n', classification_report(y_test,y_pred_test))

In [ ]:
y_prob_trainval = estimator.predict_proba(X_trainval)[:,1]
y_pred_trainval = np.where(y_prob_trainval > 0.13, 1, 0)

print('auc',roc_auc_score(y_trainval,y_prob_trainval),'\n')
print('classification report \n', classification_report(y_trainval,y_pred_trainval))

prediction

In [ ]:
# estimator = grid_search_simple.best_estimator_

In [ ]:
estimator.fit(X,y)

In [ ]:
y_pred_sub = estimator.predict_proba(X_sub)[:,1]
df_test['Best Performance'] = y_pred_sub
df_submission = df_test[['Best Performance']]

In [ ]:
df_submission.reset_index().to_csv('submission logreg2.csv',index = False)